In [10]:
!pip install snowflake['ml'] --target $nas_package_dir

  Using cached https://files.pythonhosted.org/packages/1f/46/a456b8d3ea1f03d172503166e1f3d58c67bb5b3cfd73d32bdb6c55b4d885/snowflake-0.8.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6f/55/1053b2e8e000c8aa68a6ded471a4fcc209fef77071c8d86096586a749120/snowflake_core-0.8.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/75/c9/bf69f07cf4f23fba13bd79e8e93b3b8fa7914fa8e0ef66e42332cc02b6d5/snowflake_legacy-0.8.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ed/2f/8e8862f0b77c23cdcafba56be63640f87feb8aef4421116b836f40f1f9c4/snowflake_ml_python-1.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9e/68/b3496e1d59e8c8b302df7fa5c4ce851a687b8b8ddb92d6f6254bfce77009/snowflake_snowpark_python-1.17.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a2/73/a68704750a7679d0b6d3ad7aa8d4da8e14e151ae82e6fee774e6e0d05ec8/urllib3-2.2.1-py3-none-any.whl
  Using cached https://files.pyth

In [9]:
!pip uninstall cloudpickle -y 
!pip install cloudpickle==2.2.1 --target $nas_package_dir

Uninstalling cloudpickle-3.0.0:
  Successfully uninstalled cloudpickle-3.0.0
  Using cached https://files.pythonhosted.org/packages/15/80/44286939ca215e88fa827b2aeb6fa3fd2b4a7af322485c7170d6f9fd96e0/cloudpickle-2.2.1-py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# !pip uninstall snowflake-snowpark-python -y
# !pip install snowflake-snowpark-python==1.5.1

In [1]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.types import StructType, StructField, DoubleType, StringType
import snowflake.snowpark.functions as F

In [2]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
#     "authenticator": "externalbrowser", # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
} 

In [3]:
# Make a Snowpark Connection

################################################################################################################
#  You can also use the SnowSQL Client to configure your connection params:
#  https://docs.snowflake.com/en/user-guide/snowsql-install-config.html
#
#  >>> from snowflake.ml.utils import connection_params
#  >>> session = Session.builder.configs(connection_params.SnowflakeLoginOptions()
#  >>> ).create()   
#
#  NOTE: If you have named connection params then specify the connection name
#  Example:
#  
#  >>> session = Session.builder.configs(
#  >>> connection_params.SnowflakeLoginOptions(connection_name='connections.snowml')
#  >>> ).create()
#
#################################################################################################################

# Edit the connection.json before creating the session object below
# Create Snowflake Session object
# connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Connection Established with the following parameters:
User                        : ADITYASINGH
Role                        : "ADITYASINGH"
Database                    : "FIRST_DB"
Schema                      : "PUBLIC"
Warehouse                   : "FOSFOR_INSIGHT_WH"
Snowflake version           : 8.20.10
Snowpark for Python version : 1.17.0


In [6]:
import pandas as pd
# from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session


In [8]:
data = pd.read_csv('/data/mlflow_sample_data.csv')
features = data.drop('quality', axis=1)
train_x, test_x, train_y, test_y = train_test_split(features, data['quality'], random_state=42)

In [9]:
features_pandas = pd.concat([train_x, train_y], axis=1)
features_pandas.head()

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
582,582,11.7,0.490,0.49,2.2,0.083,5.0,15.0,1.00000,3.19,0.43,9.2,5
626,626,8.8,0.600,0.29,2.2,0.098,5.0,15.0,0.99880,3.36,0.49,9.1,5
1030,1030,7.1,0.590,0.00,2.1,0.091,9.0,14.0,0.99488,3.42,0.55,11.5,7
620,620,8.3,0.540,0.24,3.4,0.076,16.0,112.0,0.99760,3.27,0.61,9.4,5
490,490,9.3,0.775,0.27,2.8,0.078,24.0,56.0,0.99840,3.31,0.67,10.6,6


In [12]:
features_pandas = features_pandas.drop("Unnamed: 0" , axis =1)
# features_pandas = map(str.upper, features_pandas.columns)
# features_pandas.head()

AttributeError: 'map' object has no attribute 'drop'

In [14]:
features_pandas = pd.concat([train_x, train_y], axis=1)
features_df = session.create_dataframe(features_pandas)


/tmp/pip_packages/snowflake/snowpark/session.py:2367: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(


In [16]:
model = XGBRegressor(learning_rate=0.1, max_depth=8, min_child_weight=1, n_estimators=100, n_jobs=1, objective="reg:squarederror", subsample=0.9500000000000001, verbosity=0)
model.fit(features_df)

RuntimeError: (2101) label_cols is not set. To read more about Snowpark ML general API differences, please refer to: https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-modeling#general-api-differences.

In [ ]:
prediction=model.predict()

In [ ]:
FEATURE_COLS = ["X1", "X2", "X3", "X4", "X5", "X6"]
LABEL_COLS = ["Y"]
OUTPUT_COLS = ["PREDICTIONS"]

# Set up data.
X, y = make_classification(
    n_samples=40000,
    n_features=6,
    n_informative=4,
    n_redundant=1,
    random_state=0,
    shuffle=True,
)

X = pd.DataFrame(X, columns=FEATURE_COLS)
y = pd.DataFrame(y, columns=LABEL_COLS)

features_pandas = pd.concat([X, y], axis=1)
features_df = session.create_dataframe(features_pandas)

# Train an XGBoost model on snowflake.
xgboost_model = XGBClassifier(
    input_cols=FEATURE_COLS,
    label_cols=LABEL_COLS,
    output_cols=OUTPUT_COLS
)

xgboost_model.fit(features_df)

# Use the model to make predictions.
predictions = xgboost_model.predict(features_df)
predictions[OUTPUT_COLS].show()